<a href="https://colab.research.google.com/github/Arvindkumar009/Assignment_1/blob/main/dataScrap_insert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas streamlit matplotlib seaborn sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade webdriver-manager selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url = "https://www.imdb.com/search/title/?release_date=2024-01-01,2024-12-31"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

movies = []
for movie in soup.select('.lister-item'):
    try:
        movies.append({
            'title': movie.h3.a.text,
            'year': movie.find('span', class_='lister-item-year').text,
            'rating': movie.strong.text if movie.strong else None,
            # Add other fields similarly
        })
    except:
        continue

pd.DataFrame(movies).to_csv('imdb_movies.csv', index=False)

In [ ]:
# imdb_scraper.py
import os
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def setup_driver():
    """Configure Chrome WebDriver with realistic settings"""
    options = webdriver.ChromeOptions()

    # Comment this out for debugging
    options.add_argument('--headless')

    # Mimic a real browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-infobars')
    options.add_argument('--start-maximized')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    # Disable automation flags
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    # Initialize driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Mask selenium detection
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver

def scrape_imdb_data(max_pages=2):
    """Scrape IMDB movie data with robust error handling"""
    driver = setup_driver()
    base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&sort=num_votes,desc"
    movies_data = []

    try:
        for page in range(1, max_pages + 1):
            url = f"{base_url}&start={((page-1)*50)+1}" if page > 1 else base_url
            print(f"\nAttempting to scrape page {page}: {url}")

            try:
                # Random delay between requests
                time.sleep(random.uniform(1, 3))

                driver.get(url)
                print("Page loaded, waiting for content...")

                # Wait for either content or consent banner
                try:
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".lister-list"))
                    )
                    print("Main content loaded")
                except TimeoutException:
                    print("Main content not found, trying alternative approach...")
                    # Alternative wait for different page structure
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".ipc-metadata-list"))
                    )

                # Handle consent banner if present
                try:
                    consent_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
                    consent_button.click()
                    print("Accepted cookies")
                    time.sleep(2)
                except NoSuchElementException:
                    pass

                # Scroll to trigger lazy loading
                print("Scrolling to load all content...")
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(random.uniform(0.5, 1.5))

                # Find all movie containers
                movie_containers = driver.find_elements(
                    By.CSS_SELECTOR,
                    ".lister-item.mode-advanced, .ipc-metadata-list-summary-item"
                )
                print(f"Found {len(movie_containers)} movie containers")

                for container in movie_containers:
                    try:
                        movie = extract_movie_data(container)
                        if movie:
                            movies_data.append(movie)
                    except Exception as e:
                        print(f"Error processing movie: {str(e)}")
                        continue

                print(f"Successfully scraped {len(movie_containers)} movies from page {page}")

            except Exception as e:
                print(f"Error processing page {page}: {str(e)}")
                continue

    except Exception as e:
        print(f"Fatal error: {str(e)}")
    finally:
        driver.quit()

    return pd.DataFrame(movies_data)

def extract_movie_data(container):
    """Extract data from a single movie container"""
    try:
        # Modern IMDB structure
        try:
            name = container.find_element(
                By.CSS_SELECTOR,
                ".lister-item-header a, .ipc-title__text"
            ).text
        except:
            return None

        # Handle different rating selectors
        rating = None
        try:
            rating = float(container.find_element(
                By.CSS_SELECTOR,
                ".ratings-imdb-rating strong, .ipc-rating-star"
            ).text.split()[0])
        except (NoSuchElementException, ValueError):
            pass

        # Handle votes
        votes = None
        try:
            votes_text = container.find_element(
                By.CSS_SELECTOR,
                "p.sort-num_votes-visible span[name='nv'], [data-testid='ratingCount']"
            ).text.replace(',', '')
            votes = int(votes_text) if votes_text.isdigit() else None
        except (NoSuchElementException, ValueError):
            pass

        # Handle duration
        duration = None
        try:
            duration_text = container.find_element(
                By.CSS_SELECTOR,
                ".runtime, .ipc-metadata-list-summary-item__li"
            ).text.split()[0]
            duration = int(duration_text) if duration_text.isdigit() else None
        except (NoSuchElementException, ValueError):
            pass

        # Handle genres
        genres = []
        try:
            genre_text = container.find_element(
                By.CSS_SELECTOR,
                ".genre, .ipc-metadata-list-summary-item__li:last-child"
            ).text
            genres = [g.strip() for g in genre_text.split(',')]
        except NoSuchElementException:
            pass

        return {
            'name': name,
            'rating': rating,
            'votes': votes,
            'duration': duration,
            'genres': ', '.join(genres)
        }

    except Exception as e:
        print(f"Error extracting movie data: {str(e)}")
        return None

def save_data(df, output_dir='data'):
    """Save data to CSV files"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if not df.empty:
        # Save all movies
        all_movies_path = os.path.join(output_dir, 'all_movies.csv')
        df.to_csv(all_movies_path, index=False)
        print(f"\nSaved all movies to {all_movies_path}")

        # Save by genre
        for genre in set(g for sublist in df['genres'].str.split(', ') for g in sublist if g):
            genre_df = df[df['genres'].str.contains(genre, na=False)]
            safe_genre = genre.lower().replace(' ', '_').replace('/', '_')
            genre_path = os.path.join(output_dir, f'{safe_genre}.csv')
            genre_df.to_csv(genre_path, index=False)

        print(f"Saved genre-specific files to {output_dir}")
    else:
        print("No data to save")

if __name__ == "__main__":
    print("Starting IMDB 2024 Movie Scraper...")
    movie_df = scrape_imdb_data(max_pages=2)

    if not movie_df.empty:
        print(f"\nSuccessfully collected {len(movie_df)} movies!")
        print(movie_df.head())
        save_data(movie_df)
    else:
        print("\nFailed to collect any movie data. Possible reasons:")
        print("- IMDB blocked the scraper (try again later)")
        print("- Website structure changed (update selectors)")
        print("- Network issues (check your connection)")

Starting IMDB 2024 Movie Scraper...

Attempting to scrape page 1: https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&sort=num_votes,desc
Page loaded, waiting for content...
Main content not found, trying alternative approach...
Scrolling to load all content...
Found 50 movie containers
Successfully scraped 50 movies from page 1

Attempting to scrape page 2: https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&sort=num_votes,desc&start=51
Page loaded, waiting for content...
Main content not found, trying alternative approach...
Scrolling to load all content...
Found 50 movie containers
Successfully scraped 50 movies from page 2

Successfully collected 100 movies!
                         name  rating votes duration genres
0           1. Dune: Part Two     8.5  None     None       
1     2. Deadpool & Wolverine     7.6  None     None       
2            3. The Substance     7.3  None     None       
3  4. Furiosa: 

In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine, types as sa_types  # Updated import
import numpy as np


def clean_data(df):
    """
    Clean the scraped movie data with robust NaN handling
    """
    # Ensure we're working with a copy
    df = df.copy()

    # Handle missing ratings - fill with median (better than mean for skewed data)
    df['rating'] = df['rating'].fillna(df['rating'].median())

    # Handle votes - fill NaN with 0 and ensure int type
    df['votes'] = pd.to_numeric(df['votes'], errors='coerce').fillna(0).astype(int)

    # Handle duration - more robust cleaning
    try:
        # First convert to numeric, coercing errors to NaN
        df['duration'] = pd.to_numeric(df['duration'], errors='coerce')

        # Replace NaN with median duration (excluding NaN values in calculation)
        duration_median = df['duration'].median(skipna=True)

        # If all durations are NaN, use a default value (e.g., 120 minutes)
        if pd.isna(duration_median):
            duration_median = 120

        df['duration'] = df['duration'].fillna(duration_median)

        # Convert to int - now safely since no NaN/inf values remain
        df['duration'] = df['duration'].astype(int)

    except Exception as e:
        print(f"Error processing duration: {e}")
        # Fallback to default duration if conversion fails
        df['duration'] = 120

    # Create duration categories - only after ensuring clean duration values
    bins = [0, 60, 120, 180, np.inf]
    labels = ['<1h', '1-2h', '2-3h', '>3h']
    df['duration_category'] = pd.cut(
        df['duration'],
        bins=bins,
        labels=labels,
        right=False
    )

    return df

def store_in_mysql(df):
    """Store cleaned data in MySQL database with error handling"""
    db_config = {
        'user': 'root',
        'password': 'admin',
        'host': 'localhost',
        'port': 3306,
        'database': 'imb_2024'
    }

    try:
        # Create SQLAlchemy engine
        connection_string = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
        engine = create_engine(connection_string)

        # Create database if not exists
        with engine.connect() as conn:
            conn.execute(f"CREATE DATABASE IF NOT EXISTS {db_config['database']}")
            conn.execute(f"USE {db_config['database']}")

        # Store data with proper SQLAlchemy types
        df.to_sql(
            name='movies',
            con=engine,
            if_exists='replace',
            index=False,
            dtype={
                'name': sa_types.String(length=255),
                'rating': sa_types.Float(),
                'votes': sa_types.Integer(),
                'duration': sa_types.Integer(),
                'genres': sa_types.String(length=255),
                'duration_category': sa_types.String(length=10)
            }
        )


        # Create views for analysis
        with engine.connect() as conn:
            # Top rated movies view
            conn.execute("""
            CREATE OR REPLACE VIEW top_rated_movies AS
            SELECT name, rating, votes, duration, genres
            FROM movies
            WHERE rating IS NOT NULL
            ORDER BY rating DESC
            LIMIT 10
            """)

            # Genre statistics view (using MySQL 8.0+ recursive CTE)
            conn.execute("""
            CREATE OR REPLACE VIEW genre_stats AS
            WITH RECURSIVE split_genres AS (
                SELECT
                    name,
                    rating,
                    votes,
                    duration,
                    SUBSTRING_INDEX(genres, ', ', 1) AS genre,
                    CASE
                        WHEN LOCATE(', ', genres) > 0
                        THEN SUBSTRING(genres, LOCATE(', ', genres) + 2)
                        ELSE ''
                    END AS remaining
                FROM movies

                UNION ALL

                SELECT
                    name,
                    rating,
                    votes,
                    duration,
                    SUBSTRING_INDEX(remaining, ', ', 1) AS genre,
                    CASE
                        WHEN LOCATE(', ', remaining) > 0
                        THEN SUBSTRING(remaining, LOCATE(', ', remaining) + 2)
                        ELSE ''
                    END AS remaining
                FROM split_genres
                WHERE remaining != ''
            )
            SELECT
                TRIM(genre) AS genre,
                COUNT(*) AS movie_count,
                AVG(rating) AS avg_rating,
                AVG(duration) AS avg_duration,
                SUM(votes) AS total_votes
            FROM split_genres
            WHERE genre != ''
            GROUP BY TRIM(genre)
            ORDER BY movie_count DESC
            """)

        print("Data successfully stored in MySQL with views created.")

    except Exception as e:
        print(f"Error storing data in MySQL: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        # Load scraped data
        df = pd.read_csv('data/all_movies.csv')

        # Clean data with robust error handling
        cleaned_df = clean_data(df)

        # Verify we have valid data before storing
        if cleaned_df.empty:
            raise ValueError("No valid data after cleaning")

        print("Data cleaning completed successfully. Sample data:")
        print(cleaned_df.head())

        # Store in MySQL
        store_in_mysql(cleaned_df)

    except Exception as e:
        print(f"Error in data processing pipeline: {str(e)}")

Data cleaning completed successfully. Sample data:
                         name  rating  votes  duration  genres  \
0           1. Dune: Part Two     8.5      0       120     NaN   
1     2. Deadpool & Wolverine     7.6      0       120     NaN   
2            3. The Substance     7.3      0       120     NaN   
3  4. Furiosa: A Mad Max Saga     7.5      0       120     NaN   
4           5. Alien: Romulus     7.1      0       120     NaN   

  duration_category  
0              2-3h  
1              2-3h  
2              2-3h  
3              2-3h  
4              2-3h  
Data successfully stored in MySQL with views created.


In [ ]:
# app.py
import streamlit as st
import pandas as pd
from sqlalchemy import create_engine

# For Streamlit >=1.18.0 use:
#@st.cache_resource
# For older versions use:
@st.experimental_singleton
def get_db_connection():
    # MySQL connection details
    db_config = {
        'user': 'root',
        'password': 'admin',
        'host': 'localhost',
        'database': 'imb_2024',
        'port': 3306
    }
    connection_string = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    return create_engine(connection_string)

# For Streamlit >=1.18.0 use:
#@st.cache_data
# For older versions use:
@st.experimental_memo
def load_data():
    try:
        engine = get_db_connection()
        movies_df = pd.read_sql("SELECT * FROM movies", engine)
        genre_stats = pd.read_sql("SELECT * FROM genre_stats", engine)
        top_rated = pd.read_sql("SELECT * FROM top_rated_movies", engine)
        return movies_df, genre_stats, top_rated
    except Exception as e:
        st.error(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

def main():
    st.title("IMDB 2024 Movie Analysis")

    # Load data
    movies_df, genre_stats, top_rated = load_data()

    if movies_df.empty:
        st.warning("No movie data available. Please check your database connection.")
        return

    # Sidebar filters
    st.sidebar.header("Filters")

    # Genre filter
    try:
        all_genres = sorted(set(g for sublist in movies_df['genres'].str.split(', ') for g in sublist if g))
        selected_genres = st.sidebar.multiselect("Select Genres", all_genres)
    except:
        selected_genres = []

    # Rating filter
    min_rating, max_rating = st.sidebar.slider(
        "Rating Range",
        min_value=0.0,
        max_value=10.0,
        value=(movies_df['rating'].min(), movies_df['rating'].max()),
        step=0.1
    )

    # Duration filter
    duration_options = ['All'] + sorted(movies_df['duration_category'].unique())
    selected_duration = st.sidebar.selectbox("Duration Category", duration_options)

    # Votes filter
    min_votes = st.sidebar.number_input("Minimum Votes",
                                      min_value=0,
                                      value=0,
                                      max_value=int(movies_df['votes'].max()))

    # Apply filters
    filtered_df = movies_df.copy()

    if selected_genres:
        filtered_df = filtered_df[filtered_df['genres'].str.contains('|'.join(selected_genres), na=False)]

    filtered_df = filtered_df[
        (filtered_df['rating'] >= min_rating) &
        (filtered_df['rating'] <= max_rating) &
        (filtered_df['votes'] >= min_votes)
    ]

    if selected_duration != 'All':
        filtered_df = filtered_df[filtered_df['duration_category'] == selected_duration]

    # Display filtered data
    st.subheader("Filtered Movies")
    st.dataframe(filtered_df)

    # Analysis tabs
    tab1, tab2, tab3 = st.tabs(["Top Movies", "Genre Analysis", "Duration Insights"])

    with tab1:
        st.subheader("Top 10 Rated Movies")
        st.dataframe(top_rated)

        st.subheader("Top 10 Most Voted Movies")
        try:
            top_voted = pd.read_sql(
                "SELECT name, votes, rating, duration, genres FROM movies ORDER BY votes DESC LIMIT 10",
                get_db_connection()
            )
            st.dataframe(top_voted)
        except Exception as e:
            st.error(f"Couldn't load most voted movies: {str(e)}")

    with tab2:
        st.subheader("Genre Statistics")
        st.dataframe(genre_stats)

        col1, col2 = st.columns(2)
        with col1:
            st.bar_chart(genre_stats.set_index('genre')['movie_count'])
        with col2:
            st.bar_chart(genre_stats.set_index('genre')['avg_rating'])

    with tab3:
        st.subheader("Duration Distribution")
        duration_counts = filtered_df['duration_category'].value_counts().sort_index()
        st.bar_chart(duration_counts)

        col1, col2 = st.columns(2)
        with col1:
            st.subheader("Shortest Movies")
            st.dataframe(filtered_df.sort_values('duration').head(10))
        with col2:
            st.subheader("Longest Movies")
            st.dataframe(filtered_df.sort_values('duration', ascending=False).head(10))

if __name__ == "__main__":
    main()

In [ ]:
pip install altair==4.2.0


  Attempting uninstall: altair
    Found existing installation: altair 5.5.0
    Uninstalling altair-5.5.0:
      Successfully uninstalled altair-5.5.0


In [ ]:
pip install --upgrade streamlit

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade streamlit

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip uninstall altair streamlit -y
!pip install altair==5.0.0 streamlit

Found existing installation: altair 4.2.0
Uninstalling altair-4.2.0:
  Successfully uninstalled altair-4.2.0
Found existing installation: streamlit 1.12.0
Uninstalling streamlit-1.12.0:
  Successfully uninstalled streamlit-1.12.0
  Using cached streamlit-1.12.0-py2.py3-none-any.whl (9.1 MB)


In [ ]:
pip uninstall streamlit altair -y

Found existing installation: streamlit 1.12.0
Uninstalling streamlit-1.12.0:
  Successfully uninstalled streamlit-1.12.0
Found existing installation: altair 5.5.0
Uninstalling altair-5.5.0:
  Successfully uninstalled altair-5.5.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install streamlit altair==4.2.0

  Using cached streamlit-1.12.0-py2.py3-none-any.whl (9.1 MB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip uninstall -y streamlit
!pip install streamlit

Found existing installation: streamlit 1.12.0
Uninstalling streamlit-1.12.0:
  Successfully uninstalled streamlit-1.12.0
  Using cached streamlit-1.12.0-py2.py3-none-any.whl (9.1 MB)


In [ ]:
import altair
print("Altair version:", altair.__version__)  # Should be 4.2.0

Altair version: 4.2.0
